In [14]:
NR1_2 =sc.read_h5ad('F:/spatial/HIP/NR1_2/spatial_transfer_digital_layer_15.h5ad')

In [15]:
NR2_2 =sc.read_h5ad('F:/spatial/HIP/NR2_2/spatial_transfer_digital_layer_15.h5ad')

In [16]:
DR1_2 =sc.read_h5ad('F:/spatial/HIP/DR1_2/spatial_transfer_digital_layer_15.h5ad')
DR2_2 =sc.read_h5ad('F:/spatial/HIP/DR2_2/spatial_transfer_digital_layer_15.h5ad')

In [27]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import zscore

# 目标基因列表
target_genes = ['Fibcd1','Prox1','Iyd']

slices_dict = {
    'NR1_2r': NR1_2,
    'NR2_2r': NR2_2,
    'DR1_2r': DR1_2,
    'DR2_2r': DR2_2
}

# 图形布局：行数 = 4（切片数），列数 = 9（目标基因数）
n_rows = len(slices_dict)
n_cols = len(target_genes)
fig, axs = plt.subplots(n_rows, n_cols, figsize=(n_cols * 2.5, n_rows * 3), constrained_layout=True)

# 遍历每个切片（行）和每个目标基因（列）
for i, (slice_name, adata_slice) in enumerate(slices_dict.items()):
    subset = adata_slice[adata_slice.obs['digital_layer'].between(1, 10)]
    
    # 使用旋转后的坐标，假设存储在 obsm['X_rotated']
    coords = subset.obsm['stereo']
    x_rot = coords[:, 0]
    y_rot = coords[:, 1]
    
    for j, gene in enumerate(target_genes):
        ax = axs[i, j]
        # 检查目标基因是否存在
        if gene not in subset.var_names:
            ax.text(0.5, 0.5, f"{gene}\n不存在", ha="center", va="center", fontsize=8)
            ax.axis("off")
            continue

        # 获取该基因在 'imputed' 层的表达数据，并转换为一维数组
        expr = subset[:, gene].layers['imputed'].toarray().flatten()
        # 标准化：先 Z-score，再 Min-Max 归一化到 [0, 1]
        expr_z = zscore(expr)
        expr_norm = (expr_z - expr_z.min()) / (expr_z.max() - expr_z.min())
        
        # 绘制散点图，颜色根据归一化后的表达值
        sc = ax.scatter(x_rot, y_rot, c=expr_norm, cmap='magma_r', s=0.5)
        #ax.set_title(f"{slice_name}\n{gene}", fontsize=8)
        ax.axis("off")
        # 为每个子图添加独立的竖直 colorbar
        #cbar = fig.colorbar(sc, ax=ax, orientation='vertical', fraction=0.05, pad=0.05)
        #cbar.ax.tick_params(labelsize=6)

#plt.suptitle("Rotated Spatial Expression of Target Genes", fontsize=16, y=1.02)
#plt.show()
output_path = "F:/spatial/HIP/figures/HIP_imputation.pdf"
plt.savefig(output_path, format='pdf', bbox_inches='tight')
plt.close()
print(f"PDF saved as {output_path}")


PDF saved as F:/spatial/HIP/figures/HIP_imputation.pdf
